In [1]:
# Nelder-Mead simplex search
%matplotlib inline
%run Trajectory_Simulation.ipynb

# i'm sorry for global vars...
global allvectors, allobjval
allvectors = []               # array for all design vecs, global variable
allobjfun = []                # array for tracking objective function evaluations

/home/cory/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Functions of Merit
We chose to abstract all of the functions used within the merit function for increased flexibility, ease of reading, and later utility.

* **objective** is arbitrarily constructed. It is
    * normalized (by a somewhat arbitrary constant) to bring it into the same range as constraints,
    * squared to reward (or punish) relative to the distance from nominal value, and
    * divided by two so that our nominal value is 0.5 instead of 1.0.
* **exact** is less arbitrarily constructed. It is
    * squared to be horizontally symmetric (which could also be obtained by absolute value),
    * determined by the distance from a constant, and
    * divided by two is for aesthetic value.
* **exterior** is not particularly arbitrary. It is
    * boolean so that we can specify whether it is minimizing or maximizing its variable,
    * 0 when the inequality is satisfied, otherwise it is just as punishing as **exact**.
* **barrier** comes in two flavors, one of which is not used here. It is
    * boolean so that we can specify whether it is a lower or an upper bound,
    * completely inviolable, unlike exact and exterior penalties.
    
Technically logarithmic barrier functions allow negative penalties (i.e. rewards), but since we use upper and lower altitude barriers, it is impossible that their sum be less than 0. If the optimizer steps outside of the apogee window, the barrier functions can attempt undefined operations (specifically, taking the logarithm of a negative number), so some error handling is required to return an infinite value in those cases. Provided that the initial design is within the feasible region, the optimizer will not become disoriented by infinite values.

In [2]:
# all of our comparisons are ratios instead of subtractions because
# it's normalized, instead of dependent on magnitudes of variables and constraints

def objective_additive(var, cons):
    return np.linalg.norm(var - cons)**2 / 2

# minimize this, **2 makes it well behaved w.r.t. when var=cons
def objective(var, cons):
    return (var/cons)**2 / 2

# **2 because i like it more than abs(), but that also works
def exact(var, cons):
    return (var/cons - 1)**2 / 2

# this is your basic exterior penalty, either punishes for unfeasibility or is inactive
def exterior(var, cons, good_if_less_than=False):
    if good_if_less_than:
        return max(0, var/cons - 1)**2 / 2
    else:
        return max(0, -(var/cons - 1))**2 / 2

# this barrier function restricts our objective function to the strictly feasible region
# make rockets great again, build that wall, etc, watch out for undefined operations
def barrier(var, cons, int_point=False, good_if_less_than=True):
    global dbz
    try: # just in case we accidentally leave feasible region
        if not int_point:
            if good_if_less_than:
                return -log(-(var/cons - 1))
            else:
                return -log(var/cons - 1)
        elif int_point:
            def interior(g): return 1/g # in case we don't like logarithms, which is a mistake
            if good_if_less_than:
                return -interior(var/cons - 1)
            else:
                return -interior(-(var/cons - 1))
    except:
        return float('inf') # ordinarily, this is bad practice since it could confuse the optimizer
                            # however, since this is a barrier function not an ordinary penalty, i think it's fine

## Optimization Problem
Given a design vector $x$ and the iteration number $n$ our merit function **f** runs a trajectory simulation and evaluates the quality of that rocket. We keep track of each design and its merit value for later visualization, hence why global variables are used.

We run an iterative sequence of optimization routines for the win. We use the Euclidean distance in the design space between successive optimal designs to decide when it is no longer worth continuing, around the 6th decimal place.

In [3]:
# this manages all our constraints
# penalty parameters: mu -> 0 and rho -> infinity 
def penalty(mu, rho):
    b = [
        ]
    eq = []
    ext = [
          ]
    return mu*sum(b) + rho*(sum(eq) + sum(ext))

# Pseudo-objective merit function
# x is array of design parameters, n is index of penalty and barrier functions
# print blocks are sanity checks so i'm not staring at a blank screen and can see what various tweaks actually do
def cost(x, nominal):
    global allvectors, allobjfun
    # get trajectory data
    sim = trajectory(144.589, 2.871, 74610.004, throttle_window, min_throttle, rcs_mdot, rcs_p_e, rcs_p_ch, 
                        ballast, root, tip, sweep, span, thickness, airfrm_in_rad,
                          OF, p_ch, T_ch, ke, MM,
                          [0, 0, x[0], x[1], False, 0, 0, 0, 0, 0, 0, False], 
                          0.05, True, 0.045, True, True)
    
    # either minimize the distance from nominal impact point
    if nominal != ' ':
        #nominal = np.array([nominal, kludge])
        obj_func = objective_additive(sim.impact, nominal)
    # or maximize distance from launch point
    else:
        obj_func = - objective_additive(sim.impact, sim.env.launch_pt)
    
    # add objective and penalty functions
    merit_func = obj_func
    allvectors.append(x) # maintains a list of every design, side effect
    allobjfun.append(merit_func)
    return merit_func

# we want to iterate our optimizer for theoretical "convergence" reasons (given some assumptions)
# n = number of sequential iterations
def iterate(func, x_0, n, nominal):
    x = x_0 #[:3] # initial design vector, stripping off throttling variables for now
    designs = []
    for i in range(n):
        print("Iteration " + str(i+1) + ":")
        # this minimizer uses simplex method
        res = minimize(func, x, args=(nominal), method='nelder-mead', options={'disp': True, 'xatol': 0.05, 'fatol': 0.05})
        x = res.x # feed optimal design vec into next iteration
        
        designs.append(res.x)   # we want to compare sequential objectives 
                                # so we can stop when convergence criteria met
    return x

def breed_rockets(func, nominal):
    res = differential_evolution(func=func, bounds=[(0, 360), (-25, 1)], args=((nominal)),
                                 strategy='best1bin', popsize=80, mutation=(.1, .8), recombination=.05,
                                 updating='immediate', disp=True, atol=0.05, tol=0.05,
                                 polish=True,workers=-1)
    return res.x

## Optimization Information and Graphing
It is to the benefit of our intuition if we can visualize the design space and our final trajectory. This block of code simply provides all of the relevant information from a trajectory as text, and displays useful graphs.

# Top-Level of Optimization Routine
Here's where the magic happens. This code block runs the iterative optimization, provides details from our optimized trajectory, uses the OpenRocket interface to make a model rocket and engine for higher-fidelity analysis, and then displays visuals.

In [4]:
# Results, this is the big boi function
if __name__ == '__main__':
    x0 = np.array([180, -30])
    # feed initial design into iterative optimizer, get most (locally) feasible design
    #x = iterate(f, x0, 1, nominal = np.array([34.49847331, -106.97472628]))
    #x = iterate(f, x0, 1, nominal = (' ', None))
    #x = breed_rockets(f, ' ')
    # probe design space, darwin style. if design space has more than 3 dimensions, you need this. takes forever.
    #res = breed_rockets(f)
    
    print("Optimization done!")

Optimization done!


In [5]:
if False and __name__ == '__main__':
# Rename the optimized output for convenience
    az_perturb    = x[0]
    el_perturb = x[1]
    
    # get trajectory info from optimal design
    sim = trajectory(144.589, 2.871, 74610.004, throttle_window, min_throttle, rcs_mdot, rcs_p_e, rcs_p_ch, 
                        ballast, root, tip, sweep, span, thickness, airfrm_in_rad,
                          OF, p_ch, T_ch, ke, MM,
                          [0, 0, az_perturb, el_perturb, False, 0, 0, 0, 0, 0, 0, True], 
                          0.05, False, 0.04, True, False)
    
    print("Azimuth Perturbation:", az_perturb)
    print("Elevation Perturbation:", el_perturb)
    print("Launch point", sim.env.launch_pt)
    print("Impact point", sim.impact)
    print()
    
    textlist = print_results(sim, False)
    # draw pretty pictures of optimized trajectory
    rocket_plot(sim.t, sim.alt, sim.v, sim.a, sim.thrust,
                sim.dyn_press, sim.Ma, sim.m, sim.p_a, sim.drag, sim.throttle, sim.fin_flutter, sim, False)
    
    # get/print info about our trajectory and rocket
    for line in textlist:
        print(line)
    
    # draw more pretty pictures, but of the optimizer guts
    #design_grapher(allvectors)

In [ ]:
# Results, this is the big boi function
if __name__ == '__main__':
    x0 = np.array([0, -10])
    # feed initial design into iterative optimizer, get most (locally) feasible design
    x = iterate(cost, x0, 1, nominal = np.array([32.913926, -106.363603]))
    #x = iterate(f, x0, 1, nominal = None)
    # probe design space, darwin style. if design space has more than 3 dimensions, you need this. takes forever.
    #x = breed_rockets(cost, nominal = np.array([32.913926, -106.363603]))
    
    print("Optimization done!")

Iteration 1:


/home/cory/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [ ]:
if __name__ == '__main__':
# Rename the optimized output for convenience
    az_perturb    = x[0]
    el_perturb = x[1]
    
    # get trajectory info from optimal design
    sim = trajectory(144.589, 2.871, 74610.004, throttle_window, min_throttle, rcs_mdot, rcs_p_e, rcs_p_ch, 
                        ballast, root, tip, sweep, span, thickness, airfrm_in_rad,
                          OF, p_ch, T_ch, ke, MM,
                          [0, 0, az_perturb, el_perturb, False, 0, 0, 0, 0, 0, 0, False], 
                          0.05, False, 0.04, True, False)
    
    print("Azimuth Perturbation:", az_perturb)
    print("Elevation Perturbation:", el_perturb)
    print("Launch point", sim.env.launch_pt)
    print("Impact point", sim.impact)
    print()
    
    textlist = print_results(sim, False)
    # draw pretty pictures of optimized trajectory
    rocket_plot(sim.t, sim.alt, sim.v, sim.a, sim.thrust,
                sim.dyn_press, sim.Ma, sim.m, sim.p_a, sim.drag, sim.throttle, sim.fin_flutter, sim, False, None, None)
    
    # get/print info about our trajectory and rocket
    for line in textlist:
        print(line)
    
    # draw more pretty pictures, but of the optimizer guts
    #design_grapher(allvectors)